In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dataset import Dataset
import numpy as np

In [ ]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [ ]:
data = Dataset(name='imdb', dirname='preprocess/IMDB/')

In [ ]:
labellist = ['sentiment']
data.generate_labels(labellist, len(labellist), 'binary')
data.save_on_metric = 'roc_auc'
data.metrics_type = 'classifier'

In [ ]:
train_data = data.filter_data_length(data.get_data('train', structured=False), 95)
test_data = data.filter_data_length(data.get_data('test', structured=False), 95)

In [ ]:
vc = [0] * data.vocab.vocab_size
for i, v in data.vocab.idx2word.items() :
    vc[i] = v

In [ ]:
from PatientVec.Experiments.modifiable_config_exp import attention_configs
avg_attn_config = attention_configs[0](data, structured=False, args=args)

In [ ]:
avg_attn_config['model']['embedder']['type'] = 'elmo_embedder'
del avg_attn_config['model']['embedder']['embedding_file']
avg_attn_config['model']['embedder']['elmo_options'] = {
    'options_file' : '../../elmo_2x2048_256_2048cnn_1xhighway_options.json',
    'weight_file' : '../../elmo_2x2048_256_2048cnn_1xhighway_weights.hdf5',
    'vocab_to_cache' : vc,
    'scalar_mix_parameters' : [0.3, 0.3, 0.3]
}

In [ ]:
avg_attn_config['training_config']['common']['bsize'] = 64

In [ ]:
from PatientVec.models.Vanilla import ClassificationTrainer as BasicCT
from PatientVec.trainer import Trainer, Evaluator

In [ ]:
import logging.config
logging.config.dictConfig({
    'version': 1,
    'disable_existing_loggers': True
})

In [ ]:
trainer = Trainer(BasicCT, avg_attn_config, _type=data.metrics_type, display_metrics=args.display)
trainer.train(train_data, test_data, save_on_metric=data.save_on_metric)

In [ ]:
outputs = trainer.model.evaluate(test_data.mock(200))

In [ ]:
for n, v in trainer.model.model.named_parameters() :
    print(n, v)